#### Transformers

In [ ]:
import pandas as pd
import numpy as np
train= pd.read_csv("AllTrain.csv")
test = pd.read_csv("AllTest.csv")
from umap import UMAP
umap_fit = UMAP(n_components=12,n_neighbors=24,random_state=0)
new = pd.concat([train,test])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(new.drop(columns=['Labels','y'])),np.array(new['y']),test_size=0.4)
X_train = umap_fit.fit_transform(X_train)
X_test = umap_fit.transform(X_test)
# X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
# X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
y_train = np.where(y_train == -1, 0, y_train)
y_test = np.where(y_test == -1, 0, y_test)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report
from time import time

# Parameters
N = 12  # number of features
M_TRAIN = X_train.shape[0]  # number of training examples
M_TEST = X_test.shape[0]  # number of test examples
BATCH = 2  # batch size
EPOCH = 10 # number of epochs

# Prepare data (assume X_train and X_test are already defined)
X_train = X_train.reshape(-1, 1, N)  # Reshape to (samples, timesteps, features)
X_test = X_test.reshape(-1, 1, N)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# Define a transformer model
def create_transformer_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    # Attention block
    x = layers.MultiHeadAttention(num_heads=4, key_dim=2)(inputs, inputs)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)  # Skip connection
    x = layers.GlobalAveragePooling1D()(x)
    
    # Feedforward network
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dense(16, activation='relu')(x)
    
    # Output layer
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs, outputs)
    return model

# Create the transformer model
model = create_transformer_model((1, N))

# Compile the model
model.compile(loss='binary_crossentropy', metrics=['accuracy'],
              optimizer=keras.optimizers.Adam(learning_rate=0.001))


# Train the model
start = time()
History = model.fit(
    X_train, y_train,
    epochs=EPOCH,
    batch_size=BATCH,
    validation_data=(X_test, y_test),
    shuffle=True,
    callbacks=[keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=1, factor=0.5),
               keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)]
)
print(f'Training completed in {time() - start:.2f} secs')

# Evaluate the model
train_loss, train_acc = model.evaluate(X_train, y_train, batch_size=BATCH)
test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=BATCH)
print(f'train accuracy = {round(train_acc * 100, 4)}%')
print(f'test accuracy = {round(test_acc * 100, 4)}%')

# Get predictions
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)  # Binarize predictions

# Generate classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_classes))

#### LSTM

In [ ]:
import pandas as pd
import numpy as np
train= pd.read_csv("AllTrain.csv")
test = pd.read_csv("AllTest.csv")
from umap import UMAP
umap_fit = UMAP(n_components=12,n_neighbors=24,random_state=0)
new = pd.concat([train,test])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(new.drop(columns=['Labels','y'])),np.array(new['y']),test_size=0.2)
X_train = umap_fit.fit_transform(X_train)
X_test = umap_fit.transform(X_test)
# X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
# X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
y_train = np.where(y_train == -1, 0, y_train)
y_test = np.where(y_test == -1, 0, y_test)
from tensorflow.keras.utils import to_categorical
X_train = X_train.reshape(-1, 1, N)  # Reshape to (samples, timesteps, features)
X_test = X_test.reshape(-1, 1, N)
# # After splitting:
# X_train, X_test, y_train, y_test = train_test_split(...)

# Before training:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Now you can safely train


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
from time import time

LAYERS = [8, 16, 8, 1]                # number of units in hidden and output layers
M_TRAIN = X_train.shape[0]           # number of training examples
M_TEST = X_test.shape[0]             # number of test examples
N = 12                                # number of features
T = 1                                  # Set T to 1 for single timestep per sample
BATCH = 2                             # batch size
EPOCH = 10                          # number of epochs
LR = 0.1                              # learning rate of the gradient descent
LAMBD = 3e-2                          # lambda in L2 regularization
DP = 0.5                              # dropout rate
RDP = 0.0                             # recurrent dropout rate
print(f'layers={LAYERS}, train_examples={M_TRAIN}, test_examples={M_TEST}')
print(f'batch = {BATCH}, timesteps = {T}, features = {N}, epochs = {EPOCH}')
print(f'lr = {LR}, lambda = {LAMBD}, dropout = {DP}, recurr_dropout = {RDP}')

model = Sequential()
model.add(LSTM(units=LAYERS[0], input_shape=(T, N),
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(3e-2), recurrent_regularizer=l2(3e-2),
               dropout=0.0, recurrent_dropout=0.0,
               return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(units=LAYERS[1], return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(units=LAYERS[2], return_sequences=False))
model.add(BatchNormalization())
model.add(Dense(units=2, activation='softmax'))  # Adjust for softmax

# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
              optimizer=Adam(learning_rate=0.001))  # Adjust learning rate

# Train the model
start = time()
History = model.fit(
    X_train, y_train,  # Use balanced data and categorical labels
    epochs=EPOCH,
    batch_size=BATCH,
    validation_data=(X_test, y_test),
    shuffle=True,
    callbacks=[ReduceLROnPlateau(monitor='loss', patience=1, factor=0.5), 
               EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True)]
)
print(f'Training completed in {time() - start:.2f} secs')

# Evaluate
train_loss, train_acc = model.evaluate(X_train, y_train, batch_size=BATCH)
test_loss, test_acc = model.evaluate(X_test, y_test, batch_size=BATCH)
print(f'train accuracy = {round(train_acc * 100, 4)}%')
print(f'test accuracy = {round(test_acc * 100, 4)}%')

# Classification report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Get class predictions
# print(classification_report(y_test, y_pred_classes))
from sklearn.metrics import classification_report

# Convert one-hot y_test to class labels
y_test_classes = np.argmax(y_test, axis=1)

# Print classification report
print(classification_report(y_test_classes, y_pred_classes))
